# Support Vector Machine (SVM) For Detecting Defective Parts

#### Erin Rasmussen, Kevin Thomas, Mike Wells, Sam Little

This notebook uses the [`scikit-learn`](http://scikit-learn.org/) library to implement a Support Vector Machine (SVM) classifier with various kernels. The SVM classifier is used to detect defective mechanical parts in a manufacturing process. The image data we used is from the ["Casting Product Image Data For Quality Inspection"](https://www.kaggle.com/ravirajsinh45/real-life-industrial-dataset-of-casting-product) dataset, and contains 8646 images used for quality inspection that we used to train the SVM.

First, we import the necessary libraries.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os
import shutil
import json
from tqdm.auto import tqdm

We declare global variables below

In [ ]:
DATA_PATH = './casting_512x512/casting_512x512/'
OUTPUT_PATH = './out/'
TRAIN_TEST_SPLIT = 0.8
RESIZE_SHAPE = (64, 64)
PRINT_ALL_INFO = False

Load image data, flattening the images, shuffling them, and splitting them into training and test sets.

In [ ]:
def load_image_data():
  data = []
  labels = []
  print('Loading OK images...')
  for filename in tqdm(os.listdir(f'{DATA_PATH}/ok_front')):
    img = Image.open(f'{DATA_PATH}/ok_front/{filename}')
    data.append(img)
    labels.append(0)
  print('Loading defective images...')
  for filename in tqdm(os.listdir(f'{DATA_PATH}/def_front')):
    img = Image.open(f'{DATA_PATH}/def_front/{filename}')
    data.append(img)
    labels.append(1)
  return data, labels

def resize_images(data, shape):
  resized_data = []
  print(f'Resizing images to {shape}...')
  for img in tqdm(data):
    resized_data.append(img.resize(shape))
  return resized_data

def prepare_image_data(data, labels):
  images = []
  print('Preparing images...')
  for img in tqdm(data):
    img = np.array(img)
    img = img / 255.0
    img[img == 0] = 0.0001
    images.append(img)
  data = np.array(images)
  original_shape = data[0].shape
  data = data.reshape(data.shape[0], -1)
  shuffled = list(zip(data, labels))

  np.random.shuffle(shuffled)
  train = shuffled[:int(len(shuffled) * TRAIN_TEST_SPLIT)]
  test = shuffled[int(len(shuffled) * TRAIN_TEST_SPLIT):]

  train_data = np.array([i[0] for i in train])
  train_labels = np.array([i[1] for i in train])
  test_data = np.array([i[0] for i in test])
  test_labels = np.array([i[1] for i in test])

  return train_data, train_labels, test_data, test_labels, original_shape

Show a random image to make sure everything is working and retain sanity

In [ ]:
data, labels = load_image_data()
data = resize_images(data, RESIZE_SHAPE)
train_data, train_labels, test_data, test_labels, original_shape = prepare_image_data(data, labels)

print('Random sample of training data:')
random_image = np.random.randint(0, len(train_data))
image = train_data[random_image].reshape(original_shape)
plt.imshow(image)
label = train_labels[random_image]
type = 'OK' if label == 0 else 'DEFECTIVE'
plt.title(f'TYPE: {type}')
plt.show()

Declare kernels we wish to test, and feed each into an `sklearn` SVM, fitting it to the training data. Accuracy, confusion matrices, and other metrics are generated for each kernel.

In [ ]:
kernels_to_test = ['linear', 'poly', 'rbf', 'sigmoid']
Cs_to_test = [0.01, 0.1, 1.0, 10.0, 100.0]
gammas_to_test = ['scale', 'auto']
image_sizes_to_test = [16, 32, 64, 128, 256, 512]

tests_to_run = []
for image_size in image_sizes_to_test:
  for kernel in kernels_to_test:
    for C in Cs_to_test:
      for gamma in gammas_to_test:
        tests_to_run.append({
          'name': f'{kernel}_{C}_{gamma}_{image_size}',
          'kernel': kernel,
          'C': C,
          'gamma': gamma,
          'image_size': image_size,
        })

results = []
last_size = None

for test in tests_to_run:
  size = (test['image_size'], test['image_size'])
  if last_size != size:
    resized_images = resize_images(data, size)
    train_data, train_labels, test_data, test_labels, original_shape = prepare_image_data(resized_images, labels)
  last_size = size
  print(f'Test: {test["name"]}')
  svm = SVC(
    kernel=test['kernel'],
    C=test['C'],
    gamma=test['gamma'],
    verbose=True
  )
  svm.fit(train_data, train_labels)
  predictions = svm.predict(test_data)
  accuracy = accuracy_score(test_labels, predictions)
  conf_matrix = confusion_matrix(test_labels, predictions)
  report = classification_report(test_labels, predictions)
  results.append({
    'name': test['name'],
    'C': test['C'],
    'gamma': test['gamma'],
    'kernel': test['kernel'],
    'accuracy': accuracy,
    'image_size': RESIZE_SHAPE,
    'confusion_matrix': conf_matrix.tolist(),
    'classification_report': report,
  })

For each kernel, we save the confusion matrix, accuracy, and other metrics to the output directory.

In [ ]:
for result in results:
  result_path = f'{OUTPUT_PATH}{result["kernel"]}/{result["image_size"]}/{result["C"]}/{result["gamma"]}'
  # if path doesn't exist, create it
  if not os.path.exists(result_path):
    os.makedirs(result_path)

  # save confusion matrix
  plt.figure(figsize=(10, 10))
  plt.imshow(result['confusion_matrix'], interpolation='nearest', cmap=plt.cm.Blues)
  plt.title(f'Confusion Matrix for {result["name"]}\nAccuracy: {result["accuracy"] * 100:.2f}%\nImage Size: {RESIZE_SHAPE}')
  plt.colorbar()
  tick_marks = np.arange(2)
  plt.xticks(tick_marks, ['OK', 'DEFECTIVE'], rotation=45)
  plt.yticks(tick_marks, ['OK', 'DEFECTIVE'])
  plt.tight_layout()
  plt.ylabel('True Label')
  plt.xlabel('Predicted Label')
  plt.savefig(f'{result_path}/confusion_matrix.png')

  if PRINT_ALL_INFO:
    plt.show()
    print(f'Confusion Matrix saved to {result_path}/confusion_matrix.png')

    print(f'Accuracy for {kernel} kernel: {result["accuracy"]}')
    print(f'Classification report for {kernel} kernel:')
    print(result['classification_report'])

  # save results to JSON
  with open(f'{result_path}/{result["name"]}.json', 'w') as f:
    result = {
      'name': result['name'],
      'C': result['C'],
      'gamma': result['gamma'],
      'kernel': result['kernel'],
      'accuracy': result['accuracy'],
      'image_size': result['image_size'],
      'confusion_matrix': result['confusion_matrix'],
      'confusion_matrix_image': f'{result_path}/confusion_matrix.png',
      'classification_report': result['classification_report']
    }
    json.dump(result, f)

Now we rate each result based on the accuracy of the classifier.

In [ ]:
results.sort(key=lambda x: x['accuracy'], reverse=True)
print("Best results:")
for index, result in enumerate(results):
  print(f'{index + 1}. {result["name"]} - {result["accuracy"] * 100:.2f}%')